# Whatsapp Sentimemt Analysis
## Proyecto Tópicos en Ciencias de la Computación

### Integrantes
- Rodrigo Heiremans
- Tomás Lagos
- Nicolás Errázuriz

### Librerías
- python 3.8.10
- numpy 1.20.3
- nltk 3.7
- lime 0.2.0.1
- spacy 3.5.1
- gcsfs 2023.3.0
- protobuf 3.20.3


## Inicialización NLTK Sentiment Analysis

In [1]:
!apt-get update
!apt-get install python-dev #hunspell
!apt-get install libhunspell-dev #hunspell

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,408 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,197 kB]
Get:14 http://ar

In [2]:
!pip install hunspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for hunspell: filename=hunspell-0.5.5-cp310-cp310-linux_x86_64.whl size=71758 sha256=7be4df0edf13a3afdc6737b1c96369f2d4583e86f103f4b42e8c800d4cf51782
  Stored in directory: /root/.cache/pip/wheels/2e/f3/bd/bdce223532ee8aa5345e14e0d6e7ba06cbbaff8767cefe1ec8
Successfully built hunspell


In [10]:
!wget https://raw.githubusercontent.com/wooorm/dictionaries/master/dictionaries/es-MX/index.aff
!wget https://raw.githubusercontent.com/wooorm/dictionaries/master/dictionaries/es-MX/index.dic
!mv /content/index.aff /content/es_CL.aff
!mv /content/index.dic /content/es_CL.dic

--2023-05-22 16:45:30--  https://raw.githubusercontent.com/wooorm/dictionaries/master/dictionaries/es-MX/index.aff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158014 (154K) [text/plain]
Saving to: ‘index.aff’

index.aff           100%[===================>] 154.31K  --.-KB/s    in 0.003s  

2023-05-22 16:45:30 (55.4 MB/s) - ‘index.aff’ saved [158014/158014]

--2023-05-22 16:45:30--  https://raw.githubusercontent.com/wooorm/dictionaries/master/dictionaries/es-MX/index.dic
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710101 (6

In [11]:
import nltk
nltk.download('vader_lexicon') # Obligatorio para usar SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

import hunspell # Nos ayudará con los diccionarios en español
# Buscar diccionarios en carpeta de instalacion
# Más diccionarios de idiomas disponibles en: https://github.com/wooorm/dictionaries/tree/master/dictionaries
diccionario = hunspell.HunSpell('/content/es_CL.dic','/content/es_CL.aff')


from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
def corregir_palabras(corrector, palabras, agregarPrimero=[]):   
    #codificacion = corrector.get_dic_encoding()   # obtenemos la codificacion para usarla luego

    # agregamos las palabras aleatorias al diccionario
    for palabra in agregarPrimero:
        corrector.add(palabra)

    # autocorreccion de palabras
    corregida = []
    for p in palabras:
        ok = corrector.spell(p)   # verificamos ortografia
        if not ok:
            sugerencias = corrector.suggest(p)
            if len(sugerencias) > 0:  # hay sugerencias
                # tomamos la  mejor sugerencia(decodificada a string)
                mejor_sugerencia = sugerencias[0]   
                corregida.append(mejor_sugerencia)
            else:
                corregida.append(p)  # no hay ninguna sugerecia para la palabra
        else:
            corregida.append(p)   # esta palabra esta corregida

    return corregida


def corregir_oracion(corrector,string):
    oracion_fixed=''
    if len(string)>0:
        oracion_dirty=string.split()
        oracion_fixed=corregir_palabras(corrector, oracion_dirty,[''])
        oracion_fixed=' '.join(oracion_fixed)
    else:
      print('error')
    return oracion_fixed

In [13]:
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace('\t', ' ')
    ReviewText = ReviewText.str.replace('\r', ' ')
    ReviewText = ReviewText.str.replace('\n', ' ')
    return (ReviewText)    

In [8]:
def get_polarity():
  for i in range(len(df)):#len(df)
      try:
          temp=df['RelatoFixedTranslated'][i]    
          # translation= translate_client.translate(temp,target_language='en',source_language='es') # Already translated
          # translation=translation['translatedText'] # Already translated
          p=sid.polarity_scores(str(temp))['compound']
          neg=sid.polarity_scores(str(temp))['neg']
          neu=sid.polarity_scores(str(temp))['neu']
          pos=sid.polarity_scores(str(temp))['pos']
          df.loc[i,'polarity']=p
          df.loc[i,'neg']=neg
          df.loc[i,'neu']=neu
          df.loc[i,'pos']=pos
          #df.loc[i,'translated']=str(translation) # Already translated
          print(i,end=' - ')
          #sleep(0.1+random.random()/2) # Already translated
      except Exception as e:
          print('Error {}:{}'.format(i,e))

In [9]:
def spell_checker_spanish():
    try:
        df.drop('Código', axis=1, inplace=True) # No viene al caso
        df.drop('Nombre', axis=1, inplace=True) # Anonimos
        df.drop('Número', axis=1, inplace=True) # No viene al caso
    except:
        pass

    df['polarity']=0
    df['neg']=0
    df['neu']=0
    df['pos']=0
    df['translated']=0

    for i in range(len(df)):#len(df)
        try:
            temp=df['Relato'][i]
            fixed=corregir_oracion(diccionario,temp)
            #fixed=preprocess(str(fixed))
            df.loc[i,'RelatoFixed']=fixed
            print(i,end=' - ')

        except Exception as e:
            print('Error {}:{}'.format(i,e))

## Preparación de DATA

## Código de Clases desde aquí

In [ ]:
from nltk.corpus import product_reviews_1
camera_reviews = product_reviews_1.reviews('Canon_G3.txt')

reviews = []

for review in camera_reviews:
    sentences = []
    for sentence in review.sents():
        text = " ".join(sentence)
        sentences.append(text)
    document = " ".join(sentences)
    reviews.append(document)


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
# It's also possible to try with a stemmer or to mix a stemmer and a lemmatizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def tokenize(document):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [lemmatizer.lemmatize(t.lower()) for t in tokenizer.tokenize(sentence) if t.lower() not in stop_words and len(t) > 2]
        words += tokens
    
    text = ' '.join(words)
    
    return text

In [ ]:
corpus = []

for review in reviews:
    document = tokenize(review)
    corpus.append(document)


In [ ]:
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() # compound in [-1,1]

Ver documentación de vader en: https://www.nltk.org/api/nltk.sentiment.vader.html

In [ ]:
label = []
for i in range(len(corpus)):
    if analyzer.polarity_scores(corpus[i])['compound'] > 0.2: 
        label.append('Positive') # positive sentiment
    elif analyzer.polarity_scores(corpus[i])['compound'] < -0.2:
        label.append('Negative') # negative sentiment
    else:
        label.append('Neutral') # neutral sentiment

In [ ]:
import pandas as pd

df = pd.DataFrame(corpus, columns = ['review'])
df['polarities'] = label
df

,review,polarities
0,recently purchased canon powershot extremely s...,Positive
1,yep first digital camera toy software engineer...,Positive
2,extensive research comparing different megapix...,Positive
3,bought canon month ago say satisfied taken hun...,Positive
4,camera one full day say wonderful photo qualit...,Positive
5,positive slr like programming exposure control...,Positive
6,camera wonderful set feature lcd screen pull r...,Positive
7,recent price drop made best bargain digital ca...,Positive
8,recommend unreservedly powershot potential buy...,Positive
9,else say camera work make photograph work want...,Positive


# Supervised sentiment analysis (training)

In [ ]:
# load in dataset and separate by the __label__ classifier in the text file
data = pd.read_csv('gs://nlp_amazon_data/train.ft.txt', sep="__label__", header = None)

<ipython-input-7-78044fd3cd00>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('gs://nlp_amazon_data/train.ft.txt', sep="__label__", header = None)


In [ ]:
data.drop(0, inplace=True, axis=1)
data['sentiment'] = data[1].str[0]
data[1] = data[1].str[2:]
data = data.rename(columns={1: 'review'})
data

,review,sentiment
0,Stuning even for the non-gamer: This sound tra...,2
1,The best soundtrack ever to anything.: I'm rea...,2
2,Amazing!: This soundtrack is my favorite music...,2
3,Excellent Soundtrack: I truly like this soundt...,2
4,"Remember, Pull Your Jaw Off The Floor After He...",2
...,...,...
3599995,Don't do it!!: The high chair looks great when...,1
3599996,"Looks nice, low functionality: I have used thi...",1
3599997,"compact, but hard to clean: We have a small ho...",1
3599998,what is it saying?: not sure what this book is...,1


In [ ]:
sample = data.sample(n=100000)

!python3 -m spacy download en_core_web_sm

In [ ]:
import string
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.blank("en") # Create a blank pipeline of a given language class
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"

def preprocessing(text):
  text = text.lower()
  text = re.sub(REGX_USERNAME, ' ', text)
  tokens = [token.text for token in nlp(text)]
  tokens = [t for t in tokens if t not in STOP_WORDS and t not in string.punctuation and len(t) > 2]
  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)

sample["text_clean"] = sample["review"].apply(preprocessing)
sample.head()

,review,sentiment,text_clean
1669564,Awesome!: These are Great! Bought these for my...,2,awesome great bought daughter boyfriend loves ...
3314648,Shame on you Lewin!: Corrupting the integrity ...,1,shame lewin corrupting integrity amazon review...
1226503,ultimate christmas c d volumn 3: here we go ag...,2,ultimate christmas volumn great christmas favo...
709998,Stand has a major defect: I bought this monito...,1,stand major defect bought monitor months ago s...
360808,"Track After Track, On Repeat, Delicious to the...",2,track track repeat delicious ears feet lies li...


In [ ]:
sample

,review,sentiment,text_clean
1669564,Awesome!: These are Great! Bought these for my...,2,awesome great bought daughter boyfriend loves ...
3314648,Shame on you Lewin!: Corrupting the integrity ...,1,shame lewin corrupting integrity amazon review...
1226503,ultimate christmas c d volumn 3: here we go ag...,2,ultimate christmas volumn great christmas favo...
709998,Stand has a major defect: I bought this monito...,1,stand major defect bought monitor months ago s...
360808,"Track After Track, On Repeat, Delicious to the...",2,track track repeat delicious ears feet lies li...
...,...,...,...
1765260,Not Recommended: Many of the recipes in this b...,1,recommended recipes book include ingredients b...
2508426,Their Greatest!: This is George Michael/Wham's...,2,greatest george michael wham greatest album tr...
3098618,Cadburys Flake Case of 24: Very good product. ...,2,cadburys flake case good product family loves ...
1203438,Can server the purpose: I bought two of these ...,1,server purpose bought items small work suggest...


In [ ]:
dataset = list(sample[["text_clean", "sentiment"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:75000]  # 75%
dev_data = dataset[75000:90000] # 15%
test_data = dataset[90000:] # 10%

In [ ]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        doc.cats["POS"] = label == '2'
        doc.cats["NEG"] = label == '1'
        db.add(doc)
    
    db.to_disk(outfile)
convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

In [ ]:
!python3 -m spacy init config --lang en --pipeline textcat --optimize efficiency --force config.cfg

2023-03-16 14:29:55.174794: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-16 14:29:55.690500: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:29:55.690563: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:29:55.690573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-16 14:29:56.644307: W tensorf

Ver documentación de config en: https://spacy.io/usage/training#quickstart

Ver documentación de architectures en: https://spacy.io/api/architectures

In [ ]:
!python3 -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

2023-03-16 14:29:59.199420: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-16 14:29:59.711462: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:29:59.711518: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:29:59.711524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-16 14:30:00.632075: W tensorf

In [ ]:
!python3 -m spacy evaluate ./model/model-best/ ./test.spacy

2023-03-16 14:34:21.918688: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-16 14:34:22.434323: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:34:22.434376: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:34:22.434383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-16 14:34:23.339304: W tensorf

In [ ]:
texts = ["This movie is unnecessarily long. At times it gets boring and hard to follow.", "I regretted ever purchasing or making order on this platform."]
nlp = spacy.load("./model/model-best")
for text in texts:
    doc = nlp(preprocessing(text))
    print(doc.cats,  "-",  text)

{'POS': 0.08944917470216751, 'NEG': 0.9105508327484131} - This movie is unnecessarily long. At times it gets boring and hard to follow.
{'POS': 0.3397209644317627, 'NEG': 0.6602790355682373} - I regretted ever purchasing or making order on this platform.
